In [83]:
from datetime import datetime as dt
import pandas as pd
import numpy as np

In [84]:
def convert_to_datetime(series_row, date_format):
    if str(series_row) == 'nan':
        return float('nan')
    return dt.strptime(series_row, date_format)
    

def convert_to_month(series_row):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    else:
        return str(series_row.month)

def convert_to_year(series_row):
    if str(series_row) == 'NaT' or str(series_row)== 'nan':
        return float('nan')
    else:
        return str(series_row.year)
    
    
def get_month_year_col(df, date_column, date_format):
    df[date_column+'_datetime'] = df[date_column].apply(convert_to_datetime, date_format=date_format)
    df[date_column+'_month'] = df[date_column+'_datetime'].apply(convert_to_month)
    df[date_column+'_year'] = df[date_column+'_datetime'].apply(convert_to_year)
    return df


def date_and_filter(df, date_format, date_col, interest_var, fac_id):
    #add datetime column
    df = get_month_year_col(df, date_col, date_format)
    
    #filter year
    year = dt.strptime('2006/12/31',"%Y/%m/%d")
    df = df[df['SETTLEMENT_ENTERED_DATE_datetime'] > year ]
    
    #filter needed
    date_time = date_col + '_datetime'
    date_year = date_col + '_year'
    df = df[interest_var + [fac_id] + [date_time] + [date_year]]
    return df


In [85]:
violhist = pd.read_csv('ICIS-AIR_VIOLATION_HISTORY.csv')

In [86]:
violhist.shape

(59256, 14)

In [87]:
fces = pd.read_csv('ICIS-AIR_FCES_PCES.csv')

In [88]:
fces.shape

(1371977, 9)

In [89]:
violhist = violhist[violhist.ENF_RESPONSE_POLICY_CODE == 'HPV']

In [90]:
violhist.shape

(35489, 14)

In [91]:
violhist = get_month_year_col(violhist, 'HPV_DAYZERO_DATE', '%m-%d-%Y') 

In [92]:
fces = get_month_year_col(fces, 'ACTUAL_END_DATE', '%m-%d-%Y') 

In [93]:
fces.columns

Index(['PGM_SYS_ID', 'ACTIVITY_ID', 'STATE_EPA_FLAG', 'ACTIVITY_TYPE_CODE',
       'ACTIVITY_TYPE_DESC', 'COMP_MONITOR_TYPE_CODE',
       'COMP_MONITOR_TYPE_DESC', 'ACTUAL_END_DATE', 'PROGRAM_CODES',
       'ACTUAL_END_DATE_datetime', 'ACTUAL_END_DATE_month',
       'ACTUAL_END_DATE_year'],
      dtype='object')

In [94]:
violhist.columns

Index(['PGM_SYS_ID', 'ACTIVITY_ID', 'AGENCY_TYPE_DESC', 'STATE_CODE',
       'AIR_LCON_CODE', 'COMP_DETERMINATION_UID', 'ENF_RESPONSE_POLICY_CODE',
       'PROGRAM_CODES', 'PROGRAM_DESCS', 'POLLUTANT_CODES', 'POLLUTANT_DESCS',
       'EARLIEST_FRV_DETERM_DATE', 'HPV_DAYZERO_DATE', 'HPV_RESOLVED_DATE',
       'HPV_DAYZERO_DATE_datetime', 'HPV_DAYZERO_DATE_month',
       'HPV_DAYZERO_DATE_year'],
      dtype='object')

In [95]:
year1 = dt.strptime('2007/01/01',"%Y/%m/%d")
year2 = dt.strptime('2016/12/31',"%Y/%m/%d")

In [96]:
violhist = violhist[violhist.HPV_DAYZERO_DATE_datetime >= year1]
violhist = violhist[violhist.HPV_DAYZERO_DATE_datetime <= year2]

In [97]:
violhist.shape

(12273, 17)

In [98]:
fces = fces[fces.ACTUAL_END_DATE_datetime >= year1]
fces = fces[fces.ACTUAL_END_DATE_datetime <= year2]

In [99]:
fces.shape

(687130, 12)

In [100]:
violhist = violhist.drop_duplicates(['PGM_SYS_ID', 'HPV_DAYZERO_DATE'], keep='first')

In [101]:
violhist.shape

(11224, 17)

In [102]:
fces = fces.drop_duplicates(['PGM_SYS_ID','ACTUAL_END_DATE'], keep='first')

In [103]:
fces.shape

(599365, 12)

In [104]:
merged_df = pd.merge(violhist, fces, how='right', left_on=['PGM_SYS_ID', 'HPV_DAYZERO_DATE'], right_on=['PGM_SYS_ID','ACTUAL_END_DATE'])

In [105]:
merged_df.shape

(599365, 28)

In [124]:
merged_df.head()

,PGM_SYS_ID,ACTIVITY_ID_x,AGENCY_TYPE_DESC,STATE_CODE,AIR_LCON_CODE,COMP_DETERMINATION_UID,ENF_RESPONSE_POLICY_CODE,PROGRAM_CODES_x,PROGRAM_DESCS,POLLUTANT_CODES,...,STATE_EPA_FLAG,ACTIVITY_TYPE_CODE,ACTIVITY_TYPE_DESC,COMP_MONITOR_TYPE_CODE,COMP_MONITOR_TYPE_DESC,ACTUAL_END_DATE,PROGRAM_CODES_y,ACTUAL_END_DATE_datetime,ACTUAL_END_DATE_month,ACTUAL_END_DATE_year
0,CT0000000900508907,3.400302e+09,State,CT,NaN,CT000A0000090050890700012,HPV,CAAFESOP,Federally-Enforceable State Operating Permit -...,300000329,...,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,03-17-2010,"CAAFESOP, CAASIP",2010-03-17 00:00:00,3,2010
1,CT0000000901100038,3.400302e+09,State,CT,NaN,CT000A0000090110003800165,HPV,CAASIP,State Implementation Plan for National Primary...,300000005,...,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,10-01-2009,CAASIP,2009-10-01 00:00:00,10,2009
2,CT0000000900900016,3.400302e+09,State,CT,NaN,CT000A0000090090001600076,HPV,CAANSPS CAANSR CAASIP CAATVP,New Source Performance Standards New Source Re...,300000329,...,S,INS,Inspection/Evaluation,FOO,FCE On-Site,04-26-2010,"CAANSPS, CAANSR, CAASIP, CAATVP",2010-04-26 00:00:00,4,2010
3,MA0000002511900766,3.400303e+09,State,MA,NaN,MA000A0000251190076690009,HPV,CAASIP CAATVP,State Implementation Plan for National Primary...,300000005,...,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,09-13-2011,"CAASIP, CAATVP",2011-09-13 00:00:00,9,2011
4,NH0000003301390006,3.400303e+09,State,NH,NaN,NH000A0000330139000600003,HPV,CAAFESOP CAASIP,Federally-Enforceable State Operating Permit -...,300000005,...,S,INS,Inspection/Evaluation,PFF,PCE Off-Site,04-06-2007,CAASIP,2007-04-06 00:00:00,4,2007


In [126]:
print(merged_df.ACTIVITY_ID_x.count(), merged_df.HPV_DAYZERO_DATE.count())

2450 2450


In [108]:
merged = merged_df[['PGM_SYS_ID', 'ACTIVITY_ID_x', 'ACTUAL_END_DATE_year']]

In [109]:
merged.ACTIVITY_ID_x

0         3.400302e+09
1         3.400302e+09
2         3.400302e+09
3         3.400303e+09
4         3.400303e+09
5         3.400305e+09
6         3.400305e+09
7         3.400307e+09
8         3.400308e+09
9         3.400308e+09
10        3.400309e+09
11        3.400308e+09
12        3.400308e+09
13        3.400308e+09
14        3.400308e+09
15        3.400308e+09
16        3.400308e+09
17        3.400317e+09
18        3.400316e+09
19        3.400318e+09
20        3.400312e+09
21        3.400309e+09
22        3.400309e+09
23        3.400309e+09
24        3.400309e+09
25        3.400309e+09
26        3.400310e+09
27        3.400310e+09
28        3.400309e+09
29        3.400309e+09
              ...     
599335             NaN
599336             NaN
599337             NaN
599338             NaN
599339             NaN
599340             NaN
599341             NaN
599342             NaN
599343             NaN
599344             NaN
599345             NaN
599346             NaN
599347     

In [110]:
merged.ACTIVITY_ID_x.fillna(0, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:3295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [111]:
merged.ix[merged.ACTIVITY_ID_x > 1, 'ACTIVITY_ID_x'] = 1

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [112]:
merged.head()

,PGM_SYS_ID,ACTIVITY_ID_x,ACTUAL_END_DATE_year
0,CT0000000900508907,1.0,2010
1,CT0000000901100038,1.0,2009
2,CT0000000900900016,1.0,2010
3,MA0000002511900766,1.0,2011
4,NH0000003301390006,1.0,2007


In [113]:
merged.shape

(599365, 3)

In [114]:
merged_group = merged.groupby(['PGM_SYS_ID', 'ACTUAL_END_DATE_year']).sum().reset_index()

In [115]:
merged_group.head()

,PGM_SYS_ID,ACTUAL_END_DATE_year,ACTIVITY_ID_x
0,020000000003605590,2015,0.0
1,020000003400500100,2014,0.0
2,020000003400546143,2010,0.0
3,020000003400710001,2015,0.0
4,020000003400900010,2013,0.0


In [116]:
merged_group.rename(columns={'ACTUAL_END_DATE_year': 'Year', 'ACTIVITY_ID_x': 'Outcome'}, inplace=True)

In [117]:
merged_group.head()

,PGM_SYS_ID,Year,Outcome
0,020000000003605590,2015,0.0
1,020000003400500100,2014,0.0
2,020000003400546143,2010,0.0
3,020000003400710001,2015,0.0
4,020000003400900010,2013,0.0


In [118]:
merged_group.Outcome.value_counts()

0.0     338007
1.0       1747
2.0        142
3.0         42
4.0         24
5.0         11
6.0          6
9.0          4
7.0          3
8.0          3
14.0         1
11.0         1
Name: Outcome, dtype: int64

In [119]:
merged_group.shape

(339991, 3)

In [120]:
merged_group.head()

,PGM_SYS_ID,Year,Outcome
0,020000000003605590,2015,0.0
1,020000003400500100,2014,0.0
2,020000003400546143,2010,0.0
3,020000003400710001,2015,0.0
4,020000003400900010,2013,0.0


In [121]:
merged_group.ix[merged_group.Outcome > 1, 'Outcome'] = 1

In [122]:
final = pd.merge(merged, merged_group, how='left', left_on=['PGM_SYS_ID','ACTUAL_END_DATE_year'], right_on=['PGM_SYS_ID','Year'])

In [180]:
final = final[['PGM_SYS_ID','ACTIVITY_ID_x','Year']]

In [181]:
final.ACTIVITY_ID_x.fillna(0, inplace = True)

In [182]:
final.ix[final.ACTIVITY_ID_x > 1, 'ACTIVITY_ID_x'] = 1

In [183]:
final.rename(columns={'ACTIVITY_ID_x': 'Outcome'}, inplace=True)

In [184]:
final

,PGM_SYS_ID,Outcome,Year
0,CT0000000900508907,1.0,2010
1,CT0000000900900016,1.0,2010
2,MA0000002511900766,1.0,2011
3,NY0000002620500175,1.0,2011
4,NY0000002600500578,1.0,2011
5,NY0000002620100208,1.0,2011
6,NY0000002630100392,1.0,2011
7,VA0000005102300039,1.0,2011
8,NY0000002620600068,1.0,2010
9,NY0000002600500572,1.0,2010


In [185]:
final.Outcome.value_counts()

0.0    188252
1.0       694
Name: Outcome, dtype: int64